<a href="https://colab.research.google.com/github/nachodele/UFV---Visualizacion/blob/main/Practicas/practica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 1 || Histórico de ventas
## 4ºB
## Ingeniería Matemática || Visualización de datos
0. Cargamos datos e importamos librerías

In [4]:
import pandas as pd
import plotly.express as px

# Cargar datos
data = pd.read_csv('datos_ejercicio_ventas.csv')

1. Como se distribuyen las ventas realizadas en:  
• Cada país






In [ ]:
# Aggregate data to get the average sales amount per country
data_agg = data.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Plot the boxplot with aggregated data
fig_pais = px.bar(data_agg, x='COUNTRY', y='AMOUNT', title="Distribución Promedio de Ventas por País")
fig_pais.show()



• Cada mes y año

In [ ]:
# Media del AMOUNT para obtener ventas totales por mes cada año
data_med = data.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Bar Plot acumulado
fig_mes = px.bar(data_med, x='MONTH', y='AMOUNT', color='YEAR', title="Distribución de Ventas por Mes y Año")
fig_mes.update_layout(
    yaxis_title="SUM OF AMOUNT"
)
fig_mes.show()


• Cada marca

In [ ]:
data_agg = data.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

# Violin Plot
fig_marca = px.bar(data_agg, x='SUBBRAND', y='AMOUNT',title="Distribución de Ventas por Marca")
fig_marca.show()

2. Cual es la tendencia y estacionalidad de:  
• Todas las ventas del país con menos ventas

In [ ]:
# Identificar país con menos ventas
pais_menos_ventas = data.groupby('COUNTRY')['AMOUNT'].sum().idxmin()
data_pais_menos = data[data['COUNTRY'] == pais_menos_ventas]

# Media del AMOUNT para obtener ventas totales por mes cada año
data_pais_menos_med = data_pais_menos.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Mostramos la tendencia con Area Plot
fig_tendencia_pais = px.area(data_pais_menos_med, x='MONTH', y='AMOUNT', color='YEAR', title=f"Tendencia de Ventas en {pais_menos_ventas}")
fig_tendencia_pais.show()

• La marca con más ventas

In [ ]:
# Identificar marca con más ventas
marca_mas_ventas = data.groupby('SUBBRAND')['AMOUNT'].sum().idxmax()
data_marca_mas = data[data['SUBBRAND'] == marca_mas_ventas]

# Media del AMOUNT para obtener ventas totales por mes cada año
data_marca_mas_med = data_marca_mas.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Evolución de ventas con Area Plot
fig_tendencia_marca = px.area(data_marca_mas_med, x='MONTH', y='AMOUNT', color='YEAR', title=f"Tendencia de Ventas para la Marca {marca_mas_ventas}")
fig_tendencia_marca.show()

3. Cuales son las predicciones hechas en España y como de buenas son.

In [32]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


# Filtrar para España y el periodo de enero de 2023 a agosto de 2024
data_spain = data[(data['COUNTRY'] == 'Spain') &
                  ((data['YEAR'] == 2023) | (data['YEAR'] == 2024)) &
                  (data['MONTH'] <= 8 if data['YEAR'].max() == 2024 else True)]

# Separar predicciones y valores reales
data_actual = data_spain[data_spain['SCENARIO'] == 'actual']
data_forecast = data_spain[data_spain['SCENARIO'] == 'AI_forecast']

# Agrupar por YEAR y MONTH para obtener el total de AMOUNT mensual
actual_monthly = data_actual.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
forecast_monthly = data_forecast.groupby(['YEAR', 'MONTH'])['AMOUNT'].mean().reset_index()

# Fusionar ambas tablas en una sola para facilitar la comparación
monthly_data = actual_monthly.merge(forecast_monthly, on=['YEAR', 'MONTH'], how='left', suffixes=('_actual', '_forecast'))

# Crear columna de fecha para el gráfico
monthly_data['DATE'] = pd.to_datetime(monthly_data[['YEAR', 'MONTH']].assign(DAY=1))

# Calcular métricas de error (e.g., Error Absoluto Medio - MAE)
monthly_data['MAE'] = abs(monthly_data['AMOUNT_actual'] - monthly_data['AMOUNT_forecast'])
mean_mae = monthly_data['MAE'].mean()  # Precisión media

# Crear gráfico de línea para visualizar la evolución de los valores reales y predicciones
fig = go.Figure()

# Línea para valores reales
fig.add_trace(go.Scatter(
    x=monthly_data['DATE'], y=monthly_data['AMOUNT_actual'],
    mode='lines+markers', name='Valores Reales',
    line=dict(color='blue')
))

# Línea para predicciones
fig.add_trace(go.Scatter(
    x=monthly_data['DATE'], y=monthly_data['AMOUNT_forecast'],
    mode='lines+markers', name='Predicciones',
    line=dict(color='orange', dash='dash')
))

# Gráfico de barras para el MAE mensual
fig.add_trace(go.Bar(
    x=monthly_data['DATE'], y=monthly_data['MAE'],
    name='Error Absoluto Medio (MAE)',
    yaxis='y2', opacity=0.5, marker=dict(color='red')
))

# Configurar el diseño del gráfico con rotación del eje X, ajuste para mostrar todos los meses y límites en el eje x
fig.update_layout(
    title="Comparación de Predicciones y Valores Reales en España (2023-2024)",
    xaxis=dict(
        title="Fecha",
        tickangle=-45,
        dtick="M1",
        range=[pd.Timestamp("2023-01-01"), pd.Timestamp("2024-08-31")]  # Limitar el rango del eje x
    ),
    yaxis=dict(title="Monto (AMOUNT)"),
    yaxis2=dict(title="Error Absoluto Medio (MAE)", overlaying='y', side='right'),
    legend=dict(x=0.1, y=0.9)
)


# Mostrar gráfico
fig.show()
